## Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn
import json
import gzip

In [3]:
%matplotlib inline

In [2]:
DATASET_PATH = '../../../datasets'
!ls $DATASET_PATH

books		  books_meta.json.gz  books_ratings.csv  lexile.pkl
books_lexile.tar  books_meta.pkl      lexile.json


### Cleaning 'Lexile Ratings'

In [25]:
lexile_files = []
for i in range(0, 675):
    file_name = DATASET_PATH+'/books/books_'+str(i)+'.txt'
    lexile_files.append(file_name)

In [39]:
frames  = []
for lex_file in lexile_files:
    df = pd.read_json(lex_file)
    df = df[['title_s','englishLexileLevel_s', 'authors_s', 'imageLinks_ss']]
    frames.append(df)
super_df = pd.concat(frames) 
super_df.rename(index=str, columns={'title_s': 'title', 'englishLexileLevel_s': 'lexile', 'authors_s':'authors', 'imageLinks_ss':'imUrl_sch' }, inplace=True)
super_df.shape

(48552, 4)

In [40]:
super_df.to_json(path_or_buf=DATASET_PATH+'/lexile.json', orient='records')

In [38]:
super_df.columns

Index([u'DRA_Level_f', u'DRA_Level_s', u'English_Guided_Reading_Level_s',
       u'Grade_Level_Equivalent_f', u'Grade_Level_Equivalent_s', u'ISBN13_s',
       u'ISBN_13_s', u'OurPrice_f', u'ProductType_s', u'authors_s',
       u'availabilityText_s', u'cartButtonText_s', u'englishLexileLevel_s',
       u'format', u'grade_ss_dp', u'grades_ss', u'id', u'imageLinks_ss',
       u'language_s', u'searchImage', u'stockAvailability', u'teachers_ss_dp',
       u'title_s', u'type_s', u'wishListErrorMessage_s', u'workTitle_s',
       u'workURL_s'],
      dtype='object')

#### Checkpoint 1 <--- execute this after restart

In [41]:
lexile_df = pd.read_json(DATASET_PATH+'/lexile.json')

### Cleaning 'Metadata'

In [19]:
%%time

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

metadata_df = getDF(DATASET_PATH+'/books_meta.json.gz');

CPU times: user 6min 5s, sys: 6.95 s, total: 6min 12s
Wall time: 6min 13s


In [20]:
metadata_df.columns

Index([u'asin', u'salesRank', u'imUrl', u'categories', u'title',
       u'description', u'related', u'price', u'brand'],
      dtype='object')

In [22]:
metadata_df = metadata_df[['asin', 'title', 'imUrl']]

In [23]:
metadata_df.to_pickle(path=DATASET_PATH+'/books_meta.pkl')

#### Checkpoint 2 <--- Execute this after restart

In [4]:
%%time

metadata_df = pd.read_pickle(DATASET_PATH+'/books_meta.pkl');

CPU times: user 1.69 s, sys: 411 ms, total: 2.1 s
Wall time: 2.12 s


### Merging Lexile and Metadata

In [35]:
print 'MetaData-> Total Titles: ', metadata_df['title'].size, '\tUnique Titles:', metadata_df['title'].unique().size

MetaData-> Total Titles:  2370585 	Unique Titles: 1860814


In [36]:
uni_meta_titles = metadata_df['title'].unique()

In [37]:
print 'Lexile-> Total Titles: ', lexile_df['title'].size, '\tUnique Titles:',lexile_df['title'].unique().size 

Lexile-> Total Titles:  48552 	Unique Titles: 45900


In [38]:
uni_lexile_titles = lexile_df['title'].unique()

In [42]:
common_titles = set(uni_meta_titles).intersection(set(uni_lexile_titles))

In [43]:
len(common_titles)

9910

In [43]:
deduplicated_metadata = metadata_df.drop_duplicates(subset=['title'])

In [44]:
deduplicated_lexile = lexile_df.drop_duplicates(subset=['title'])

In [45]:
merged_df = pd.merge(deduplicated_metadata, deduplicated_lexile, on=['title'])

In [46]:
len(merged_df)

9910

In [47]:
pd.to_pickle(merged_df, DATASET_PATH+'/merged_df.pkl')

In [48]:
merged_df.columns

Index([u'asin', u'title', u'imUrl', u'authors', u'imUrl_sch', u'lexile'], dtype='object')

In [49]:
merged_df = pd.read_pickle(DATASET_PATH+"/merged_df.pkl")